<a href="https://colab.research.google.com/github/gmarambaia/gmarambaia/blob/main/Capstone_Coursera_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linear Regression Model<p>
Regression Tree Model <p>
Neural Network Model<p>

Evaluation and selection of the Model

Lenders would also like to predict the amount of losses. Even if a loan is bad, they can recover some of the money they lent.<p>

For this part of the project, I'll need to consider loans that have `loan_status` **Charged Off**. 

In order to do it, will be necessary to create a new column called `loss`, which is the difference between `loan_amnt` and `total_pymnt`.<p>

Again the dataset will be splitted in training and validation sets using a 60:40 split.<p>

To compare their performance I will use **R-squared and RMS Error**. `loss` will be the dependent variable and all other variables except `loan_amnt`, `total_pymnt`, and `loan_status` will work as explanatory variables.

In [8]:
# importing the libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [9]:
df = pd.read_csv('df_predictions.csv')
df.head()

,loan_amnt,int_rate,installment,annual_inc,revol_bal,revol_util,total_acc,acc_open_past_24mths,total_pymnt,loan_status_bin,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,8000,6.68,245.85,41000.0,24377,51.0,29,2,8351.630000,0,0,0,1,1,0,0
1,12175,9.17,388.13,100000.0,21329,64.6,17,3,13205.910000,0,1,0,0,1,0,0
2,6400,6.92,197.38,41900.0,14936,73.2,15,1,3550.380000,1,0,0,1,0,1,0
3,12600,6.68,387.22,73800.0,9904,20.7,25,4,13125.770000,0,0,1,0,1,0,0
4,9000,8.67,284.82,82000.0,46158,77.1,29,2,9789.870001,0,1,0,0,0,0,1


In the first part of this project we attributed binary values to the varible loan_status. The status Charged Off = 1 and Fully Paid = 0. So the first thing we gonna do is drop all entries with fully paid status.

In [10]:
df['loan_status_bin'].value_counts()

0    5401
1    1077
Name: loan_status_bin, dtype: int64

We will work with only 1,077 entries. Ok let's do it.

In [11]:
#dropping entries with Fully Paid status
df.drop(df[df.loan_status_bin == 0].index, inplace=True)

In [12]:
df['loan_status_bin'].value_counts()

1    1077
Name: loan_status_bin, dtype: int64

Now, let's create our new varibale called `loss`

In [14]:
#creating the variable 'loss'
df["loss"] = (df["loan_amnt"] - df["total_pymnt"])
df.head()

,loan_amnt,int_rate,installment,annual_inc,revol_bal,revol_util,total_acc,acc_open_past_24mths,total_pymnt,loan_status_bin,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loss
2,6400,6.92,197.38,41900.0,14936,73.2,15,1,3550.38,1,0,0,1,0,1,0,2849.62
6,6000,7.89,187.72,43000.0,3102,50.9,20,3,1701.25,1,0,0,1,0,1,0,4298.75
8,10000,10.99,327.34,36000.0,9022,67.8,49,4,3594.63,1,0,0,1,0,1,0,6405.37
11,16000,18.25,580.45,65000.0,15460,58.8,17,6,8662.40,1,0,0,1,1,0,0,7337.60
16,12000,17.57,431.25,60000.0,16084,76.5,14,4,6850.96,1,0,0,1,0,1,0,5149.04


In [15]:
#dropping 'loan_amnt', 'total_pymnt' and 'loan_status_bin'
df.drop(['loan_amnt', 'total_pymnt', 'loan_status_bin'], axis=1, inplace = True)
df.head()

,int_rate,installment,annual_inc,revol_bal,revol_util,total_acc,acc_open_past_24mths,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loss
2,6.92,197.38,41900.0,14936,73.2,15,1,0,0,1,0,1,0,2849.62
6,7.89,187.72,43000.0,3102,50.9,20,3,0,0,1,0,1,0,4298.75
8,10.99,327.34,36000.0,9022,67.8,49,4,0,0,1,0,1,0,6405.37
11,18.25,580.45,65000.0,15460,58.8,17,6,0,0,1,1,0,0,7337.60
16,17.57,431.25,60000.0,16084,76.5,14,4,0,0,1,0,1,0,5149.04
